# 测量中心倾向

In [1]:
import pandas as pd

In [2]:
import os
from tiingo import TiingoClient

config = {}
config['session'] = True
config['api_key'] = os.environ['TIINGO_API_KEY']

client = TiingoClient(config)

def get_df_from_tiingo(symbol, start, end):
    df = pd.DataFrame(client.get_ticker_price(symbol,
                      startDate=start,
                      endDate=end,
                      frequency='daily'
                     ))
    df.set_index('date', inplace=True)
    df.index = pd.to_datetime(df.index)
    df.index = df.index.tz_localize('UTC')
    del df.index.name
    return df

# 算术平均值

算术平均数用于总结数字数据，并且通常是用“平均值”一词来表示的。它被定义为观测值的总和除以观测值的数量：
$$
\mu = \frac{\sum_{i=1}^N X_i}{N}
$$
其中 $X_1, X_2, \ldots, X_N$ 是观察值。

In [3]:
import scipy.stats as stats
import numpy as np

x1 = [1, 2, 2, 3, 4, 5, 5, 7]
x2 = x1 + [100]

print('Mean of x1:', sum(x1), '/', len(x1), '=', np.mean(x1))
print('Mean of x2:', sum(x2), '/', len(x2), '=', np.mean(x2))

Mean of x1: 29 / 8 = 3.625
Mean of x2: 129 / 9 = 14.3333333333


我们还定义了加权平均数，这对于我们明确定义了每个观察值应计数的次数时很有用。例如，在计算投资组合的平均价值时，说有 70% 的股票是 X 类型的比列出每一份股票的清单要方便。

加权算术平均值被定义为
$$
\sum^{n}_{i=1} w_i X_i
$$

其中 $\sum^{n}_{i=0} w_i = 1$。

# 中位数

In [4]:
print('Median of x1:', np.median(x1))
print('Median of x2:', np.median(x2))

Median of x1: 3.5
Median of x2: 4.0


# 众数

In [5]:
print('One mode of x1:', stats.mode(x1)[0][0])

def mode(l):
    counts = {}
    for e in l:
        if e in counts:
            counts[e] += 1
        else:
            counts[e] = 1
            
    maxcount = 0
    modes = {}
    
    for (key, value) in counts.items():
        if value > maxcount:
            maxcount = value
            modes = {key}
        elif value == maxcount:
            modes.add(key)
    
    if maxcount > 1 or len(l) == 1:
        return list(modes)
    
    return 'No mode'

print('All of the modes of x1:', mode(x1))

One mode of x1: 2
All of the modes of x1: [2, 5]


对于有很多不同值的数据，例如回报数据，可能不会有值超过一次出现的。这种情况下，我们可以像创建直方图时一样取值，找到数据集的众数，其中每个值用它的箱的名称替换。这样，我们可以发现哪些元素最常出现。

In [6]:
start = '2014-01-01'
end = '2015-01-01'
pricing = get_df_from_tiingo('SPY', start, end).loc[:,'adjClose']
returns = pricing.pct_change()[1:]
print('Mode of returns:', mode(returns))

Mode of returns: No mode


In [7]:
hist, bins = np.histogram(returns, 20)
maxfreq = max(hist)
print('Mode of bins:', [(bins[i], bins[i+1]) for i, j in enumerate(hist) if j == maxfreq])

Mode of bins: [(-0.0012500271794381734, 0.0011116733213610841)]


# 几何平均数

算术平均数使用加法，而几何平均数使用乘法：
$$
G = \sqrt[n]{X_1X_2\ldots X_n}
$$
对于 $X_i \ge 0$ 我们可以使用对数来写做一个算术平均值：
$$
\ln{G} = \frac{\sum^{n}_{i=1}\ln X_i}{n}
$$

几何平均值总是小于等于算术平均值。

In [8]:
print('Geometric mean of x1:', stats.gmean(x1))
print('Geometric mean of x2:', stats.gmean(x2))

Geometric mean of x1: 3.09410402498
Geometric mean of x2: 4.55253458762


当有负数时如何计算几何平均值呢？这个问题在资产回报的例子中很好解决，因为我们的值最小为 $-1$。我们可以将 $R_t$ 加上 $1$ 得到 $1 + R_t$，这样最后的公式为：
$$
R_G = \sqrt[T]{(1+R_1)\ldots (1+R_T)} - 1
$$

In [9]:
# 为数组中每个值加 1
ratios = returns + np.ones(len(returns))
R_G = stats.gmean(ratios) - 1
print('Geometric mean of returns:', R_G)

Geometric mean of returns: 0.000541695567009


由于几何平均数的定义，如果整个时间段内的回报率恒定且等于 $R_G$，证券的最终价格会与回报率为 $R_1,\ldots, R_T$ 的情况相同。

In [10]:
T = len(returns)
init_price = pricing[0]
final_price = pricing[T]
print('Initial price:', init_price)
print('Final price:', final_price)
print('Final price as computed with R_G:', init_price * (1 + R_G) ** T)

Initial price: 168.11174661
Final price: 192.58886649
Final price as computed with R_G: 192.58886649


# 调和平均数

调和平均值比其他类型的平均值更少使用。它被定义为
$$
H = \frac{n}{\sum_{i=1}^n \frac{1}{X_i}}
$$


与几何平均值一样，我们可以将调和均值重写为算术平均值。调和平均值的倒数是观察值的倒数的算术平均值：
$$
\frac{1}{H} = \frac{\sum_{i=1}^n \frac{1}{X_i}}{n}
$$

对于非负数 $X_i$ 的调和均值不大于算术均值，并且只有当所有观察值相等时它们才相等。

In [11]:
print('Harmonic mean of x1:', stats.hmean(x1))
print('Harmonic mean of x2:', stats.hmean(x2))

Harmonic mean of x1: 2.55902513328
Harmonic mean of x2: 2.86972365624


当数据可以使用比率自然表达时，可以使用调和均值。例如，在平均成本的策略中，固定金额定期花在股票。股票价格越高，那么采用该策略的投资者购买的股票越少。他们为股票支付的平均（算术平均）金额时价格的调和平均值。

# 点估计可能会骗人

由于它将整个分布合并为一个数字，所以均值隐藏了大量信息。你应该小心确保你没有由于汇总数据丢失重要信息。

## 潜在分布可能是错误的

即使你使用了正确的指标来衡量均值与离散度，但如果潜在分布不符合你的想法，它们就没有意义。例如，使用标准差来衡量一个事件的频率通常会假设它是正态分布。尽量不要去假设分布，除非必须，这种情况下，你应该严格检查数据是否符合你假设的分布。